* Name of Project - 
* Tools used - 4 to 5
* Description
  * bullet 1
  * bullet 2
  * bullet 3
  * bullet 4

In [71]:
import plotly.express as px

In [ ]:
# import re
# import time
# import tabulate

In [1]:
import pandas as pd
import numpy as np

from get_data import get_nba_teams, get_nba_players
from clean_data import add_and_adjust_cols

nba_teams = get_nba_teams()
nba_players = get_nba_players(nba_teams)
nba_df = add_and_adjust_cols(nba_players)


How many players are currently on an NBA roster?  

In [2]:
nba_df.shape[0]

520

Who is the youngest team in the NBA?  In other words, which team has the youngest roster on average? 

In [3]:
nba_df.groupby('team')['current_age'].mean().sort_values(ascending=True).head(5)

team
Oklahoma City Thunder    23.406845
San Antonio Spurs        23.833712
Orlando Magic            23.962583
Indiana Pacers           24.222496
Houston Rockets          24.420977
Name: current_age, dtype: float64

In [ ]:
'''
YOUNGEST TEAM IN NBA - METHOD #2: 
subtracts each player's DoB from right now which creates a timedelta object, then calculate a year figure in int format
'''
nba_df['current_age'] = nba_df.dob.apply(lambda x: (dt.datetime.now() - x).days / 365.25)
youngest_teams = nba_df.groupby('team')['current_age'].mean().sort_values(ascending=True).head(5)
youngest_teams = youngest_teams.sort_values(ascending=True).head(5)


In [58]:

# oldest teams
nba_df.groupby('team')['current_age'].mean().sort_values(ascending=False).head()

# lightest teams
lightest = nba_df.groupby('team')['weight_int'].mean().sort_values(ascending=True).head()

# heaviest teams
heaviest = nba_df.groupby('team')['weight_int'].mean().sort_values(ascending=False).head()

# tallest teams
tallest = nba_df.groupby('team')['height_in'].mean().sort_values(ascending=False).head()

# shortest teams
shortest = nba_df.groupby('team')['height_in'].mean().sort_values(ascending=True).head()

# Top 10 colleges by representation
#nba_df.pivot_table(index='college', aggfunc='size').sort_values(ascending=False).head(10)

In [68]:
colleges_ten = nba_df.pivot_table(index='college', aggfunc='size').sort_values(ascending=False).head(10)

In [73]:
colleges_ten

college
Duke              25
Kentucky          25
UCLA              12
Michigan          12
Arizona           11
USC               11
Kansas            10
Gonzaga           10
Texas             10
North Carolina    10
dtype: int64

In [69]:
type(colleges_ten)

pandas.core.series.Series

In [77]:
fig = px.bar(colleges_ten, 
        hover_data=['value'],
        labels={'value': '# of alums in NBA'})


In [78]:
fig.show()

In [ ]:
nba_df = pd.read_csv('nba_df.csv')

In [ ]:
'''
YOUNGEST TEAM IN NBA - METHOD #1: 
converts date of birth to nanoseconds in order to run the mean, then converts back to dt
'''
nba_df['dob_ns'] = nba_df.dob.values.astype(np.int64)
pd.to_datetime(nba_df.groupby(by='team')['dob_ns'].mean()).sort_values(ascending=False)


In [ ]:
nba_df.head(1)

In [ ]:
# >>> df.groupby('ID')[['Val1','Val2']].corr()
#nba_df.head(1)
#nba_df.groupby('team')[['current_age', 'weight_int']].corr()
nba_df.corr()